In [1]:
import awkward as ak
import numpy as np
import hist as hs
from coffea import processor, hist as chs
from coffea.nanoevents.methods import vector, candidate
from numba import jit

from coffea.nanoevents import BaseSchema
import mplhep as hep
import matplotlib.pyplot as plt
from datetime import date
today = date.today()

plt.style.use(hep.style.CMS)

/uscms_data/d3/aaportel/micromamba/envs/analysis-env/lib/python3.9/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [15]:
class LLP_ntuple_processor(processor.ProcessorABC):
    """
    This class is used to process the ntuples created by the LLP ntuple producer.
    """
    
    
    def process(self, events):
        
        muoncut = abs(events.lepPdgId == 13)
        events['muonE'] = events.lepE[muoncut]
        events['muonPt'] = events.lepPt[muoncut]
        events['muonEta'] = events.lepEta[muoncut]
        events['muonPhi'] = events.lepPhi[muoncut]
        events['muonPdgId'] = events.lepPdgId[muoncut]
        events['muonDZ'] = events.lepDZ[muoncut]
        events['muonLooseId'] = events.lepLooseId[muoncut]
        events['muonTightId'] = events.lepTightId[muoncut]
        
        dataset = events.metadata['dataset']
        out = {
            dataset: {
                "entries": len(events),
            },
            "cuts": {},
            "vars": {},

        }
        
        cuts = {}
        
        cuts['lead_muon_cut'] = (ak.max(events.muonPt, axis=1) == events.muonPt)
        
        cuts['muon_acc_cut'] = (ak.count(events.muonPhi, axis=1) > 0) #event level
        
        if events.metadata['dataset'] == 'signal':
            cuts['llp_acc_cut'] = (events.gLLP_csc == 1) #event level
            
            
        _ = lambda x: x
        bins = 30
        vrbls = {
            'met':               [bins,    0, 100, _,   'metEENoise'],
            'dphi':              [bins,    0,   5, _,   'cscRechitClusterMetEENoise_dPhi'],
            'timeSpread':        [bins,    0,  20, _,   'cscRechitClusterTimeSpreadWeightedAll'],
            'clsTimeWeighted':   [bins, -5.5,  13, _,   'cscRechitClusterTimeWeighted'],
            'clsSize':           [bins,    0, 130, _,   'cscRechitClusterSize'],
            'clsEta':            [bins,    0,   4, abs, 'cscRechitClusterEta'],
            'NStation10':        [bins,  -10,  10, _,   'cscRechitClusterNStation10'],
            'AvgStation10':      [bins,  -10,  10, _,   'cscRechitClusterAvgStation10'],
            #'deltaR_muon_cls':   [bins,    0,   4, _,   'cscRechitCluster_deltaR_muon_cls'],
            #'deltaEta_muon_cls': [bins,    0,   4, _,   'cscRechitCluster_deltaPhi_muon_cls'],
            #'deltaPhi_muon_cls': [bins,    0,   4, _,   'cscRechitCluster_deltaEta_muon_cls'],
        }
        
        def histmaker(var):
            out[dataset][f'before'] = hs.Hist.new.Reg(30, 0, 4, name='before', label='before').Double()
            out[dataset][f'before'].fill(ak.flatten(events.cscRechitClusterEta, axis=None))

 
        
        return out   
    
    def postprocess(self, accumulator):
        return accumulator

In [16]:
date = today.strftime("%m_%d_%y")

fileset = {}
fileset['signal']     = ['root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root',]
#fileset['background'] = ['root://cmsxrootd.fnal.gov//store/user/christiw/displacedJetMuonAnalyzer/csc/V1p171/Data2018_UL/v3/normalized/ParkingBPH4_Run2018A_goodLumi.root']
out = processor.run_uproot_job(
    fileset,
    treename="MuonSystem",
    processor_instance=LLP_ntuple_processor(),
    executor=processor.futures_executor,
    executor_args={"schema": BaseSchema, "workers": 1},
    maxchunks = 1
)

cuts = list(out['cuts'].keys())
varnames = list(out['vars'].keys())

Output()

In [24]:
print(out['signal']['before'].sum())

print(out['signal']['after'].sum())


3571.0
3560.0
